In [ ]:
import pandas
import geopandas
import shapely
import pathlib
import rioxarray

In [ ]:
base_path = pathlib.Path(r"C:\Users\pearsonra\Documents\data\Waikanae\flood_observations")
csv_name = base_path / "Flood_Debris.csv"

In [ ]:
data = pandas.read_csv(csv_name, thousands=',')

In [ ]:
geometry = []
for i, row in data.iterrows():
    geometry.append(shapely.geometry.Point([row.X, row.Y]))
data['geometry'] = geometry

In [ ]:
geo_data = geopandas.GeoDataFrame(data, geometry='geometry', crs=2193)

In [ ]:
#geo_data.groupby('Date_').count()

In [ ]:
geo_data.rename(columns={'level_':'Z_wellington_LVD_1953'}, inplace=True)

In [ ]:
geo_data.plot()

In [ ]:
geo_data.to_file(base_path / "Flood_Debris.geojson")

# Converting from LVD to ZNVD 2016

In [ ]:
lvd_path = pathlib.Path(r"C:\Users\pearsonra\Documents\data\local_datums\lds-wellington-1953-to-nzvd2016-conversion-raster-GTiff\wellington-1953-to-nzvd2016-conversion-raster.tif")
obs_path = pathlib.Path(r"C:\Users\pearsonra\Documents\data\Waikanae\flood_observations\Flood_Debris.geojson")

In [ ]:
with rioxarray.rioxarray.open_rasterio(lvd_path, masked=True) as lvd_conv:
    lvd_conv.load()
lvd_conv = lvd_conv.copy(deep=True)

In [ ]:
obs_pts = geopandas.read_file(obs_path)

In [ ]:
obs_pts.columns

In [ ]:
Z_NZVD_2016 = []
for index, row in obs_pts.iterrows():
    Z_NZVD_2016.append(row.Z_wellington_LVD_1953 - 
                       lvd_conv.interp(x=row.X, y=row.Y).data[0])
data['Z_NZVD_2016'] = Z_NZVD_2016

In [ ]:
geo_data.to_file(base_path / "Flood_Debris.geojson")